## Homework I ML in finance


authors: Summer Negahdar, Luna Jian, Jae Jung


In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pyarrow.parquet as pq
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [2]:
gw_df= pd.read_csv("gw.csv")
lsret_df= pd.read_csv("lsret.csv", index_col=0)
FRED_df= pd.read_csv("FREDMD.csv")
largeml_df= pd.read_parquet("largeml.pq")
smallml_df= pd.read_parquet("smallml.pq")



### Question one

### Question Two

### Question Three


a. 

b. 

In [10]:
##introducing indicator column for all rows
lsret_df['indicator']= 1 

# Ensure index is datetime
lsret_df.index = pd.to_datetime(lsret_df.index)

train= lsret_df[lsret_df.index< '2004-01-01'].dropna()
test= lsret_df[lsret_df.index >= '2004-01-01'].dropna()
X_train= train.drop(columns= 'indicator')
y_train= train['indicator']
X_test= test.drop(columns= 'indicator')



In [22]:
#I am removing low-variance features for less instability

variances = X_train.var()
low_var_cols = variances[variances < 1e-6].index
if len(low_var_cols) > 0:
    print(f"Warning: Dropping {len(low_var_cols)} low-variance columns: {low_var_cols}")
    X_train = X_train.drop(columns=low_var_cols)
    X_test = X_test.drop(columns=low_var_cols)

# scaling features for LASSO and RR
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [23]:
## I am just checking for a few things bc I keep getting LAsso errors
print("Columns with zero variance:", (X_train == 0).all().sum())
print("Duplicated columns:", X_train.T.duplicated().sum())
print("NaNs in X_train_scaled:", np.isnan(X_train_scaled).sum())
print("Infs in X_train_scaled:", np.isinf(X_train_scaled).sum())

Columns with zero variance: 0
Duplicated columns: 0
NaNs in X_train_scaled: 0
Infs in X_train_scaled: 0


In [24]:
#fitting lasso and ridge
lasso = LassoCV(
    cv=5,
    random_state=42,
    max_iter=50000,
    tol=1e-2,
    alphas=np.logspace(-3, -0.5, 30)
).fit(X_train_scaled, y_train)



##I got an error message that in finance means they features are closely correlated. that is why 
# I am increasing number of iterations from 10k to 50k
# relaxing the tolerance
# changing alphas to a smaller range. 


In [21]:
## Now RR
ridge = RidgeCV(cv=5, alphas=[0.1, 1.0, 10.0]).fit(X_train_scaled, y_train)

In [25]:
# Normalize weights
def normalize_weights(coef, model_name):
    total = np.sum(np.abs(coef))
    if total < 1e-10:
        print(f"Warning: {model_name} coefficients sum to near zero; using equal weights")
        return np.ones(len(coef)) / len(coef)
    return coef / total

In [28]:
print("Lasso Coefs:", lasso.coef_)
print("Ridge Coefs:", ridge.coef_)

Lasso Coefs: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Ridge Coefs: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

so the coefficients for LAsso and ridge (which does not cancel out any coef) is zero. this suggests that there is no combination of long-short portfolios that return a flat rate of 1. and since all coefs are zero, our weights will be equally set across all of them. 

In [29]:
w_lasso = normalize_weights(lasso.coef_, "Lasso")
w_ridge = normalize_weights(ridge.coef_, "Ridge")

# Portfolio returns
port_lasso = (X_test @ w_lasso).rename('Lasso Portfolio')  # Use original X_test for returns
port_ridge = (X_test @ w_ridge).rename('Ridge Portfolio')


the formula for sharpe ratio will be

$
\text{Sharpe Ratio} = \frac{\mathbb{E}[R_p] \times 12}{\sigma(R_p) \times \sqrt{12}}
$

In [30]:

# Sharpe ratios
def compute_sharpe(returns, model_name):
    mean = returns.mean() * 12
    std = returns.std() * np.sqrt(12)
    if std < 1e-10:
        print(f"Warning: {model_name} portfolio has near-zero volatility")
        return np.nan
    return mean / std

sharpe_lasso = compute_sharpe(port_lasso, "Lasso")
sharpe_ridge = compute_sharpe(port_ridge, "Ridge")
print(f"Lasso Sharpe Ratio: {sharpe_lasso:.3f}")
print(f"Ridge Sharpe Ratio: {sharpe_ridge:.3f}")

Lasso Sharpe Ratio: 1.844
Ridge Sharpe Ratio: 1.844


as we mentioned before, since the coefs are zero and weights are equal, then the sharpe ratio for both of them will be the same. in this case it will be 1.844 . sharpe ratio measures for risk-adjusted return on the portfolio and in our case the sharpe ratio is considered "good". this implies that the diversification of portfolio will decrease the risk and increase the expected retun on our portfolio even though we failed to "optamize" using Lasso and RR. 